In [1]:
import pandas as pd
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import RSLPStemmer
nltk.download('stopwords')
nltk.download('rslp')
import matplotlib.pyplot as plt
import heapq
from google.colab import drive
import time

# Realizando a leitura do dataset e tokenização

db = pd.read_csv("https://raw.githubusercontent.com/GersonSales/rec-info/master/lab_04/results.csv")

documents = db['text']
stopwords = stopwords.words("portuguese")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [0]:
def buildIndex(documents):
  toker = RegexpTokenizer(r'\b[A-zÀ-ú\d\-\']+')
  inverted_list = {}
  n = 0

  for document in documents:
    n += 1
    tokens = toker.tokenize(document.lower())
    for token in set(tokens):
      if (token not in stopwords and len(token) >= 2):
        count = (n, tokens.count(token))
        if (token not in inverted_list.keys()):
          inverted_list[token] = []
          inverted_list[token].append(count)
        else:
          inverted_list[token].append(count)
  return inverted_list
inverted_index = buildIndex(documents)

In [3]:
inverted_index_df = pd.DataFrame.from_dict(inverted_index, orient="index")
inverted_index_df.to_csv('data.csv')
!cp data.csv drive/My\ Drive/

cp: cannot create regular file 'drive/My Drive/': No such file or directory


In [4]:
def documentAtATime(query, inverted_index, k):
  inverted_lists = []
  r = []
  for word in query.split(" "):
    if word in inverted_index.keys():
      inverted_lists.append(inverted_index[word])
  for document in range(1, len(documents)+1):
    sd = 0
    for inverted_list in inverted_lists:
      for i in inverted_list:
        if (i[0] == document):
          sd += i[1]
          break
    heapq.heappush(r, (sd, document))
  return heapq.nlargest(k, r)
    
documentAtATime("empresa", inverted_index, 10)

[(10, 142),
 (10, 40),
 (9, 143),
 (8, 22),
 (7, 134),
 (7, 21),
 (6, 30),
 (5, 136),
 (4, 126),
 (3, 243)]

In [5]:
def termAtATime(query, inverted_index, k):
  a = {}
  inverted_lists = []
  r = []
  for word in query.split(" "):
    if word in inverted_index.keys():
      inverted_lists.append(inverted_index[word])
  for inverted_list in inverted_lists:
    for post in inverted_list:
      d = post[0]
      freq = post[1]
      if (d in a.keys()):
        a[d] += freq
      else:
        a[d] = freq
  for (d, ad) in a.items():
    sd = ad
    heapq.heappush(r, (sd, d))
  return heapq.nlargest(k, r)
    
termAtATime("empresa", inverted_index, 10)

[(10, 142),
 (10, 40),
 (9, 143),
 (8, 22),
 (7, 134),
 (7, 21),
 (6, 30),
 (5, 136),
 (4, 126),
 (3, 243)]

In [0]:
queries = ["política", "presidente", "educação", "ministério", "empresa"]

In [7]:
results_document = []
results_term = []
time_results_document = []
time_results_term = []
k = 10

for query in queries:
  init_document = time.time()
  results_document.append(documentAtATime(query, inverted_index, k))
  end_document = time.time()
  time_results_document.append(end_document - init_document)
  
  init_term = time.time()
  results_term.append(termAtATime(query, inverted_index, k))
  end_term = time.time()
  time_results_term.append(end_term - init_term)

queries_df = pd.DataFrame()
queries_df['query'] = queries
queries_df['document_at_a_time'] = results_document
queries_df['term_at_a_time'] = results_term
queries_df['compare'] = queries_df.document_at_a_time == queries_df.term_at_a_time
queries_df['tempo_médio_document_at_a_time'] = time_results_document
queries_df['tempo_médio_term_at_a_time'] = time_results_term
queries_df

,query,document_at_a_time,term_at_a_time,compare,tempo_médio_document_at_a_time,tempo_médio_term_at_a_time
0,política,"[(9, 69), (7, 235), (7, 166), (6, 204), (5, 17...","[(9, 69), (7, 235), (7, 166), (6, 204), (5, 17...",True,0.001491,0.000104
1,presidente,"[(16, 166), (15, 63), (12, 151), (11, 216), (1...","[(16, 166), (15, 63), (12, 151), (11, 216), (1...",True,0.001842,0.000228
2,educação,"[(22, 221), (11, 222), (7, 130), (6, 239), (5,...","[(22, 221), (11, 222), (7, 130), (6, 239), (5,...",True,0.000669,0.000066
3,ministério,"[(8, 221), (7, 124), (7, 115), (4, 203), (3, 2...","[(8, 221), (7, 124), (7, 115), (4, 203), (3, 2...",True,0.000840,0.000057
4,empresa,"[(10, 142), (10, 40), (9, 143), (8, 22), (7, 1...","[(10, 142), (10, 40), (9, 143), (8, 22), (7, 1...",True,0.000874,0.000070
